In [1]:
import numpy as np 
import nibabel as nib 
from glob import glob 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import argparse
import numpy as np 
#import pingouin as pg
#!pip install pingouin
#import pingouin as pg
#pg.partial_corr(data=df, x='A', y='B', covar='Z')
import pandas as pd 
import os 
from numpy.testing import assert_array_almost_equal

import numpy as np
import sklearn.datasets, sklearn.decomposition
import matplotlib.pyplot as plt
import pylab
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import cut_tree, fcluster
import imageio

%autosave 5
%matplotlib inline



Autosaving every 5 seconds


In [ ]:


def to_gif(filenames, saveas): 
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(saveas, images, duration=0.75 )


def get_pcs(X, nComp=None, idComp=None, plot=None): 
    # https://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-com

    #X = FEF_cat
    print("shape: "+ str(X.shape))
    
    mu = np.mean(X, axis=0)

    pca = sklearn.decomposition.PCA()
    pca.fit(X)

    if nComp != None: 
        #nComp = 2
        Xhat = np.dot(pca.transform(X)[:,:nComp], pca.components_[:nComp,:])
        Xhat += mu

    elif idComp != None: 
        #idComp = 2
        Xhat = np.dot(pca.transform(X)[:,idComp].reshape([ X.shape[0],1]), pca.components_[idComp,:].reshape([1, X.shape[1]]))
        Xhat += mu


    data_pca = pca.transform(X)
    
    exp_var = pca.explained_variance_ratio_


    #print(Xhat[0,])
    #print(Xhat[0,].shape)
    #print(Xhat.shape)
    
    
    # return pcs, recon, loss, cumLoss, 
    return data_pca, Xhat, exp_var 



def cluster(o_array, save_path, dpi=400, size=(24,24)):
    '''
    o_array=array
    df=ids
    path='./hierarchicalClust'
    dpi=400
    size=(24,24)
    '''

    #if not os.path.exists(save_path):
    #    os.makedirs(save_path)

    print("o_array shape: {}".format(o_array.shape))

    D = np.corrcoef(o_array)

    print("D shape: {}".format(D.shape))

    len_rois = D.shape[0]

    # Compute and plot dendrogram.
    fig = pylab.figure(figsize=size, dpi=dpi)
    
    
    axdendro = fig.add_axes([0.09,0.1,0.2,0.8])
    Y = sch.linkage(D, method='centroid')
    Z = sch.dendrogram(Y, orientation='right')

    index           = Z['leaves']
    #labels          = [ '-'.join([df['r1'].iloc[l], df['r2'].iloc[l]]) for l in range(df.shape[0]) ]
    #labels_reorg    = [ labels [x] for x in index ]
    labels_reorg = ids 

    #axdendro.set_xticks([])
    #axdendro.set_yticks([])
    #axdendro.set_xticks(range(len_rois))
    #axdendro.set_yticks(range(len_rois))
    #axdendro.set_xticklabels(labels_reorg)
    axdendro.set_yticklabels(labels_reorg)

    # Plot distance matrix.
    axmatrix = fig.add_axes([0.3,0.1,0.6,0.8])

    D = D[index,:]
    D = D[:,index]

    im = axmatrix.matshow(D, aspect='auto', origin='lower')
    axmatrix.set_xticks([])
    axmatrix.set_yticks([])

    # Plot colorbar.
    axcolor = fig.add_axes([0.91,0.1,0.02,0.8])
    pylab.colorbar(im, cax=axcolor)

    # Display and save figure.
    fig.show()

    plt.savefig(os.path.join(save_path,'dendrogram.ALL.jpeg'))

    plt.close()

    return Y,Z

work_dir="/data/NIMH_scratch/kleinrl/ds003216-download/derivatives/sub-02/VASO_fun2_javier"
data_dir=work_dir+"/data"
out_dir=work_dir+"/out"
roi_dir=work_dir+"/rois"
timeseries_maindir=work_dir+"/timeseries"
plot_dir=work_dir+"/plots"


sess = glob(timeseries_maindir+"/VASO_grandmean_WITHOUT-ses-13_spc")
sess+= glob(timeseries_maindir+"/sub*_spc")
#sess = sess[0:3]

roi="FEF"
types = ["ward"] #, "centroid", "median", "weighted", "average", "complete", "single" ]

for type in types: 
    for ses in sess:
        for i in [-1, 0, 1, 2, 3, 5, 7]:

            
            plot_file = plot_dir+'/clust_'+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    
            plot_file_hist = plot_dir+'/hist_'+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    
            plot_file_voxs = plot_dir+'/voxs_'+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    


            #gm_wo_ses13=timeseries_maindir+"/VASO_grandmean_WITHOUT-ses-13_spc"
            #tss=glob(gm_wo_ses13+"/*")
            #print(tss)


            ts_FEF_2D=glob(ses+"/*"+roi+"*.2D")
            ts_FEF_1D=glob(ses+"/*"+roi+"*.1D")
            ts_FEF_2D.sort()
            ts_FEF_1D.sort()

            # ts_VIP_2D=glob(ses+"/*VIP*.2D")
            # ts_VIP_1D=glob(ses+"/*VIP*.1D")
            # ts_VIP_2D.sort()
            # ts_VIP_1D.sort()



            # VIP     = [ np.loadtxt(x) for x in ts_VIP_2D ] 
            # VIP_ind = np.concatenate([len(x)*[y] for x,y in zip(VIP, range(1,len(VIP)+1)) ])
            # VIP_cat = np.concatenate(VIP)

            FEF     = [ np.loadtxt(x) for x in ts_FEF_2D ] 
            FEF_ind = np.concatenate([len(x)*[y] for x,y in zip(FEF, range(1,len(FEF)+1)) ])
            #FEF_ind = np.concatenate([len(x)*[y] for x,y in zip(FEF, range(0,len(FEF))) ])
            FEF_cat = np.concatenate(FEF)

            # ALL_cat = np.concatenate([FEF_cat, VIP_cat])
            # ALL_ind_roi = np.concatenate([[0 for x in range(len(FEF_cat))],[1 for x in range(len(VIP_cat))]])
            # ALL_ind_lay = np.concatenate([FEF_ind, VIP_ind])

            if i == -1: 
                D = np.corrcoef(FEF_cat)
            else: 
                data_pca, Xhat, exp_var = get_pcs(FEF_cat, nComp=i)
                D = np.corrcoef(Xhat)


            #print("D shape: {}".format(D.shape))

            len_rois = D.shape[0]

                    # Compute and plot dendrogram.
                    # dpi=400
                    # size=(24,24)
                    # fig = pylab.figure(figsize=size, dpi=dpi)

            size=(12,12)
            fig = pylab.figure(figsize=size)
            
            
            
                    #fig = pylab.figure(nrows=2, ncols=5)#), dpi=dpi)
            #fig, axs = plt.subplots(ncols=5, nrows=4, figsize=(15,12))#, layout="constrained")
            #axdendro = plt.subplot(1,2,nrows=2, ncols=2)
            
            plt.title(plot_file.split('/')[-1])


            axdendro = fig.add_axes([0.09,0.1,0.2,0.8])
            Y = sch.linkage(D, method=type) # 'centroid'
            Z = sch.dendrogram(Y, orientation='right')

            index               = Z['leaves']
            color_list          = Z['color_list']
            #leaves_color_list   = Z['leaves_color_list']

            if FEF_ind.shape[0] != len(color_list):
                FEF_ind = FEF_ind[0:-1]
                
            
            index_array = np.array(index)
            
            
            chart = []
            to_plot = []
            to_plot_labs =[]
            voxs_by_color = []
            voxs_by_color_inds = []
            vox_by_color = []
            
            for u in np.unique(color_list): 
                ind_color = np.isin(color_list, u)
                lay_counts = FEF_ind[ind_color]
                
                
                vox_by_color_ind = index_array[np.where(ind_color)]
                voxs_by_color_inds.append(vox_by_color_ind)
                vox_by_color.append(FEF_cat[vox_by_color_ind])
                
                
                #print(" ")
                to_plot.append(lay_counts)
                to_plot_labs.append(u)
                
                for uu in range(8):
                    ind = np.where(lay_counts == uu)
                    
                    #print("{}    {}    {}    {}".format(u, uu,len(ind[0]),  len(ind[0])/len(ind_color)*100))
                    chart.append([u, uu,len(ind[0]),  len(ind[0])/len(ind_color)*100])

            chart = pd.DataFrame(chart)
            #to_plot = np.concatenate(to_plot)
            
            
            

            #labels_reorg = ids 

            axdendro.set_xticks([])
            axdendro.set_yticks([])
            
            # fix this need to assign labels 
            #axdendro.set_yticklabels(labels_reorg)

            # Plot distance matrix.
            axmatrix = fig.add_axes([0.3,0.1,0.6,0.8])
            #axmatrix = plt.subplot(1,1,ncols=2, nrows=2)
            
            D = D[index,:]
            D = D[:,index]



            im = axmatrix.matshow(D, aspect='auto', origin='lower')
            
            axmatrix.set_xticks([])
            axmatrix.set_yticks([])

            # Plot colorbar.
            axcolor = fig.add_axes([0.91,0.1,0.02,0.8])
            pylab.colorbar(im, cax=axcolor)

            # Display and save figure.
            fig.show()
            
            plt.savefig(plot_file)

            plt.close()
            
            
            
            
            # HIST PLOTS 
            fig = pylab.figure(figsize=size)
            plt.hist(to_plot, 7, label=to_plot_labs, histtype='bar', stacked=False, fill=True)
            plt.legend(prop={'size': 10})
            plt.title(plot_file.split('/')[-1])
            plt.savefig(plot_file_hist)
            plt.close()



            # VOX PLOTS 
            fig = pylab.figure(figsize=size)
            p = []
            for i_vox in range(len(vox_by_color)):
                voxs    = vox_by_color[i_vox]
                mu      = np.mean(voxs, axis=0)
                stdev   = np.std(voxs, axis=0)
                
                plt.plot(mu, label=to_plot_labs[i_vox])
                plt.fill_between(range(mu.shape[0]),mu-stdev,mu+stdev,alpha=.1)

            plt.legend(loc="upper right") #p, to_plot_labs, prop={'size': 10}
            plt.title(plot_file.split('/')[-1])
            plt.savefig(plot_file_voxs)
            plt.close()

            
            FEF_cat_sorted = FEF_cat[index]
            
            rois2 = [   "L_V1",
                        "L_V2",
                        "L_V3",
                        "L_V4",
                        "L_FST",
                        "L_PH",
                        "L_MS",
                        "L_LO3",
                        "L_MT",
                        "L_V4t",
                        "L_MST",
                        "L_VIP",
                        "L_VIP",
                                                
                        "L_LIPv",
                        "L_LIPd",

                        "L_7Pm",
                        "L_7m",
                        "L_7AL",
                        "L_7Am",
                        "L_7PL",
                        "L_7PC",
                        ]
            
            for roi2 in rois2:
            
                ts_ROI2_2D=glob(ses+"/*"+roi2+"*.2D")
                ts_ROI2_1D=glob(ses+"/*"+roi2+"*.1D")
                ts_ROI2_2D.sort()
                ts_ROI2_1D.sort()

                ROI2     = [ np.loadtxt(x) for x in ts_ROI2_2D ] 
                ROI2_ind = np.concatenate([len(x)*[y] for x,y in zip(ROI2, range(1,len(ROI2)+1)) ])
                ROI2_cat = np.concatenate(ROI2)
                
                
                plot_file_corr = plot_dir+'/corr_'+roi2+"_"+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    

                # len_x=FEF_cat.shape[0]
                # len_y=ROI2_cat.shape[0]
                # c = np.zeros(shape=(len_x,len_y))
                # for x in range(len_x): 
                #     for y in range(len_y):
                #         c[x,y] = np.corrcoef(FEF_cat[x], ROI2_cat[y])[0,1]
                cat = np.concatenate([FEF_cat_sorted, ROI2_cat])
                c = np.corrcoef(cat)
                
                fig = pylab.figure(figsize=size)
                p = plt.imshow(c)
                fig.colorbar(p)
                # fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)

                plt.title(plot_file.split('/')[-1])
                plt.savefig(plot_file_corr)
                plt.close()

                
                
                # SINGLE COMP 
                plot_file_corr_single = plot_dir+'/corrSingle_'+roi2+"_"+type+"_"+str(i)+"_"+ses.split('/')[-1]+"_"+roi+".jpeg"    

                len_x=FEF_cat_sorted.shape[0]
                len_y=ROI2_cat.shape[0]
                
                cc = c[:len_x,len_x:]
                
                fig = pylab.figure(figsize=size)
                p = plt.imshow(cc)
                fig.colorbar(p)
                # fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)

                plt.title(plot_file.split('/')[-1])
                plt.savefig(plot_file_corr_single)
                plt.close()
            
            
            
            
    filenames=glob(plot_dir+"/clust*"+type+"_"+str(i)+"_*FEF.jpeg")
    saveas=plot_dir+"/clust_"+type+"_"+str(i)+"_FEF.gif"
    to_gif(filenames, saveas)      
                
    filenames=glob(plot_dir+"/hist*"+type+"_"+str(i)+"_*FEF.jpeg")
    saveas=plot_dir+"/hist_"+type+"_"+str(i)+"_FEF.gif"
    to_gif(filenames, saveas)      
            
    filenames=glob(plot_dir+"/voxs*"+type+"_"+str(i)+"_*FEF.jpeg")
    saveas=plot_dir+"/voxs_"+type+"_"+str(i)+"_FEF.gif"
    to_gif(filenames, saveas)      
            
            
# plot the clustered timeseires 
#   averages
#   individual voxs 
# skew plots 
# summary plots voxs per layer 
# correlated the clustered timerseries 
#       across brian, but also nuisance timecousrses csf, resp
#       




In [ ]:
plt.hist(to_plot, 7, label=to_plot_labs, )
plt.title(plot_file.split('/')[-1])
print(to_plot)


In [ ]:


filenames=glob(plot_dir+"/clust*ward_7_*FEF.jpeg")
len(filenames)
saveas=plot_dir+"/clust_ward_7_FEF.gif"
to_gif(filenames, saveas)


In [ ]:



def get_pcs(X, nComp=None, idComp=None, plot=None): 
    # https://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-com

    #X = FEF_cat
    print("shape: "+ str(X.shape))
    
    mu = np.mean(X, axis=0)

    pca = sklearn.decomposition.PCA()
    pca.fit(X)

    if nComp != None: 
        #nComp = 2
        Xhat = np.dot(pca.transform(X)[:,:nComp], pca.components_[:nComp,:])
        Xhat += mu

    elif idComp != None: 
        #idComp = 2
        Xhat = np.dot(pca.transform(X)[:,idComp].reshape([ X.shape[0],1]), pca.components_[idComp,:].reshape([1, X.shape[1]]))
        Xhat += mu


    data_pca = pca.transform(X)
    
    exp_var = pca.explained_variance_ratio_


    #print(Xhat[0,])
    #print(Xhat[0,].shape)
    #print(Xhat.shape)
    
    
    # return pcs, recon, loss, cumLoss, 
    return data_pca, Xhat, exp_var 



In [ ]:

def seed_valuation_plots(ALL_cat, ALL_ind_roi, ALL_ind_lay, plot_name):


    # ALL_height = ALL_cat.shape[0]
    # c = np.zeros((ALL_height))
    # for i in range(ALL_height):
    #     c[i] = np.corrcoef(seed, ALL_cat[i,:])[0,1]
    
    # ALL_height = ALL_cat.shape[0]
    # cc = np.zeros((ALL_height, ALL_height))
    # for i in range(ALL_height):
    #     for j in range(ALL_height):
    #         cc[i,j] = np.corrcoef(ALL_cat[i,:], ALL_cat[j,:])[0,1]
    
    cc = np.corrcoef(ALL_cat)

    FEF_ind = np.where(ALL_ind_roi == 0 )
    FEF_means = []
    FEF_stdevs = []

    VIP_means = []
    VIP_stdevs = []

    for i in range(1,8):
        
        rows = ALL_ind_roi == 0
        cols = (ALL_ind_lay == i ) & (ALL_ind_roi == 0)


        FEF_means.append(np.mean(cc[np.ix_(rows, cols)] ))
        FEF_stdevs.append(np.std(cc[np.ix_(rows, cols)] ))

        rows = ALL_ind_roi == 0
        cols = (ALL_ind_lay == i ) & (ALL_ind_roi == 1)

        VIP_means.append(np.mean(cc[np.ix_(rows, cols)] ))
        VIP_stdevs.append(np.std(cc[np.ix_(rows, cols)] ))

        # FEF_means.append(np.mean(cc[(ALL_ind_roi == 0) & (ALL_ind_lay == i ),       ALL_ind_roi == 0]))
        # FEF_stdevs.append(np.mean(c[(ALL_ind_roi == 0) & (ALL_ind_lay == i ),       ALL_ind_roi == 0]))

        # VIP_means.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i ),        ]))
        # VIP_stdevs.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i ),       ]))


    FEF_means = np.array(FEF_means)
    FEF_stdevs = np.array(FEF_stdevs)

    VIP_means = np.array(VIP_means)
    VIP_stdevs = np.array(VIP_stdevs)





    fig, axs = plt.subplots(ncols=4, nrows=3, figsize=(15,9))#, layout="constrained")

    p = axs[0, 1].imshow(cc)
    #plt.colorbar()
    #fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)



    axs[0, 2].plot(FEF_means)
    axs[0, 2].fill_between(range(7),FEF_means-FEF_stdevs,FEF_means+FEF_stdevs,alpha=.1)

    axs[0, 3].plot(VIP_means)
    axs[0, 3].fill_between(range(7),VIP_means-VIP_stdevs,VIP_means+VIP_stdevs,alpha=.1)

    axs[0,2].title.set_text("FEF")
    axs[0,3].title.set_text("VIP")

    axs[0,2].set(ylabel="r-values")
    axs[0,3].set(ylabel="r-values")

    fig.savefig(plot_dir+"/"+plot_name)
    

i=10

data_pca, Xhat, exp_var = get_pcs(FEF_cat, nComp=i)
ALL_xhat = np.concatenate([Xhat, VIP_cat])

plot_name ='FEF_VIP_recon-nComp{}.png'.format(i)
seed_valuation_plots(ALL_xhat, ALL_ind_roi, ALL_ind_lay, plot_name)




In [ ]:

for i in range(10):
    data_pca, Xhat, exp_var = get_pcs(FEF_cat, nComp=i)
    ALL_xhat = np.concatenate([Xhat, VIP_cat])
    
    plot_name ='FEF_VIP_recon-nComp{}.png'.format(i)
    seed_valuation_plots(ALL_xhat, ALL_ind_roi, ALL_ind_lay, plot_name)
    
    
    
    data_pca, Xhat, exp_var = get_pcs(FEF_cat, idComp=i)
    ALL_xhat = np.concatenate([Xhat, VIP_cat])
    
    plot_name ='FEF_VIP_recon-idComp{}.png'.format(i)
    seed_valuation_plots(ALL_xhat, ALL_ind_roi, ALL_ind_lay, plot_name)



In [ ]:

fig, axs = plt.subplots(ncols=4, nrows=5, figsize=(10,30))#, layout="constrained")


for i in range(5):
    
    data_pca, Xhat, exp_var = get_pcs(FEF_cat, nComp=i)
    ALL_xhat = np.concatenate([Xhat, VIP_cat])
    
    plot_name ='FEF_VIP_recon-nComp{}.png'.format(i)
    #seed_valuation_plots(ALL_xhat, ALL_ind_roi, ALL_ind_lay, plot_name)
    
    

    
    cc = np.corrcoef(ALL_xhat)

    FEF_ind = np.where(ALL_ind_roi == 0 )
    FEF_means = []
    FEF_stdevs = []

    VIP_means = []
    VIP_stdevs = []

    for i in range(1,8):
        
        rows = ALL_ind_roi == 0
        cols = (ALL_ind_lay == i ) & (ALL_ind_roi == 0)


        FEF_means.append(np.mean(cc[np.ix_(rows, cols)] ))
        FEF_stdevs.append(np.std(cc[np.ix_(rows, cols)] ))

        rows = ALL_ind_roi == 0
        cols = (ALL_ind_lay == i ) & (ALL_ind_roi == 1)

        VIP_means.append(np.mean(cc[np.ix_(rows, cols)] ))
        VIP_stdevs.append(np.std(cc[np.ix_(rows, cols)] ))

    FEF_means = np.array(FEF_means)
    FEF_stdevs = np.array(FEF_stdevs)

    VIP_means = np.array(VIP_means)
    VIP_stdevs = np.array(VIP_stdevs)




    p = axs[0, 1].imshow(cc)


    axs[0, 2].plot(FEF_means)
    axs[0, 2].fill_between(range(7),FEF_means-FEF_stdevs,FEF_means+FEF_stdevs,alpha=.1)

    axs[0, 3].plot(VIP_means)
    axs[0, 3].fill_between(range(7),VIP_means-VIP_stdevs,VIP_means+VIP_stdevs,alpha=.1)

    axs[0,2].title.set_text("FEF")
    axs[0,3].title.set_text("VIP")

    axs[0,2].set(ylabel="r-values")
    axs[0,3].set(ylabel="r-values")

    fig.savefig(plot_dir+"/"+plot_name)
    

i=10

data_pca, Xhat, exp_var = get_pcs(FEF_cat, nComp=i)
ALL_xhat = np.concatenate([Xhat, VIP_cat])

plot_name ='FEF_VIP_recon-nComp{}.png'.format(i)
seed_valuation_plots(ALL_xhat, ALL_ind_roi, ALL_ind_lay, plot_name)




In [ ]:





# cc_recon = np.corrcoef(ALL_xhat)




# fig, axs = plt.subplots(ncols=4, nrows=3, figsize=(10,6), layout="constrained")

# p = axs[0, 0].imshow(cc)
# #plt.colorbar()
# fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)



# axs[0, 1].plot(FEF_means)
# axs[0, 1].fill_between(range(7),FEF_means-FEF_stdevs,FEF_means+FEF_stdevs,alpha=.1)

# axs[0, 2].plot(VIP_means)
# axs[0, 2].fill_between(range(7),VIP_means-VIP_stdevs,VIP_means+VIP_stdevs,alpha=.1)

# axs[0,1].title.set_text("FEF")
# axs[0,2].title.set_text("VIP")

# axs[0,1].set(ylabel="r-values")
# axs[0,2].set(ylabel="r-values")

# fig.savefig(plot_dir+"/"+plot_name)




In [ ]:


# JHF6673 Northcarolina


def get_pcas(X_train):
 

    scaler = MinMaxScaler()

    data_rescaled = scaler.fit_transform(X_train)

    #pca = PCA(n_components = 0.99)
    pca = PCA(n_components=30)

    pca.fit(data_rescaled)
    
    U, S, VT = np.linalg.svd(data_rescaled - data_rescaled.mean(0))
    #assert_array_almost_equal(VT[:30], pca.components_)
    
    
    data_pca = pca.transform(data_rescaled)
    exp_var = pca.explained_variance_ratio_

    rets = [] 
    # for i in range(data_pca.shape[1]): 
        
    #     recon_pc = data_pca[:,i].reshape([data_pca.shape[0],1])
    #     data_projected = pca.inverse_transform(recon_pc)

    #     #data_projected = pca.inverse_transform(data_pca)

    #     loss = np.sum((data_rescaled - data_projected) ** 2, axis=1).mean()
        
    #     ret = {"pc":i, 
    #         "var_exp":exp_var[i], 
    #         "data_projected":data_projected, 
    #         "loss":loss}
        
    #     rets.append(ret)
        
    
    data_projected_all = pca.inverse_transform(data_pca)
    data_projected_all_mean += data_rescaled.mean() 
    
    loss_all = np.sum((data_rescaled - data_projected_all) ** 2, axis=1).mean()
    
    ret = {"pc":'all', 
        "var_exp":exp_var.sum(), 
        "data_projected":data_projected_all, 
        "loss":loss_all}

    rets.append(ret)
    
    

    ncols = 3
    fig, axs = plt.subplots(ncols=ncols, nrows=2, figsize=(10,6))#, layout="constrained")

    #p = axs[0, 0].imshow(cc)
    #plt.colorbar()
    #fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)


    for i in range(ncols):
        

        axs[0, i].plot([x for x in range(data_projected_all.shape[0])], data_projected_all[:,i])
        #axs[0, i].plot([x for x in range(data_pca.shape[0])], data_pca[:,i])
        axs[0, i].plot([x for x in range(X_train.shape[0])], X_train[:,i])
        axs[0, i].plot([x for x in range(data_rescaled.shape[0])], data_rescaled[:,i])

        
        #plt.legend()

    return 
    

    return data_pca, data_projected_all, rets


data = FEF_cat.T

get_pcas(data)



In [ ]:

def get_pcas_2(data): 
    
    # data = FEF_cat.T
    
    # Subtract mean from data
    mean = np.mean(data, axis= 0)
    print("Mean ", mean.shape)
    mean_data = data - mean
    print("Data after subtracting mean ", data.shape, "\n")

        
    # Compute covariance matrix
    cov = np.cov(mean_data.T)
    #cov = np.round(cov, 2)
    print("Covariance matrix ", cov.shape, "\n")
        
    # Perform eigen decomposition of covariance matrix
    eig_val, eig_vec = np.linalg.eig(cov)
    print("Eigen vectors ", eig_vec)
    print("Eigen values ", eig_val, "\n")

    # Sort eigen values and corresponding eigen vectors in descending order
    indices = np.arange(0,len(eig_val), 1)
    indices = ([x for _,x in sorted(zip(eig_val, indices))])[::-1]
    eig_val = eig_val[indices]
    eig_vec = eig_vec[:,indices]
    print("Sorted Eigen vectors ", eig_vec)
    print("Sorted Eigen values ", eig_val, "\n")


    # Get explained variance
    sum_eig_val = np.sum(eig_val)
    explained_variance = eig_val/ sum_eig_val
    print(explained_variance)
    
    cumulative_variance = np.cumsum(explained_variance)
    print(cumulative_variance)


    # Take transpose of eigen vectors with data
    pca_data = np.dot(mean_data, eig_vec)
    print("Transformed data ", pca_data.shape)

    pca_data_1 = np.dot(mean_data, eig_vec[0,:])



    recon_data = pca_data 
    org_data = data 
    
    # Compute reconstruction loss
    #loss = np.mean(np.square(recon_data - org_data))
    loss = np.mean((recon_data - org_data)**2)
    print("Reconstruction loss ", loss)

    recon_data2 = recon_data +mean 
    
    for i in range(3):
        

        plt.plot([x for x in range(org_data.shape[0])], org_data[:,i])
        plt.plot([x for x in range(recon_data.shape[0])], recon_data2[:,i])
        
        #plt.legend()

    return 


data = FEF_cat.T

get_pcas_2(data)





In [ ]:
df = pd.DataFrame(ALL_cat.T, columns=[x for x in range(ALL_cat.shape[0])])

pg.partial_corr(data=df, x=0, y=1, covar=[2,3])




In [ ]:


def seed_valuation_plots(seed, ALL_cat, ALL_ind_roi, ALL_ind_lay, plot_name):


    ALL_height = ALL_cat.shape[0]
    c = np.zeros((ALL_height))
    for i in range(ALL_height):
        c[i] = np.corrcoef(seed, ALL_cat[i,:])[0,1]
    
    
    # ALL_height = ALL_cat.shape[0]
    # cc = np.zeros((ALL_height, ALL_height))
    # for i in range(ALL_height):
    #     for j in range(ALL_height):
    #         cc[i,j] = np.corrcoef(ALL_cat[i,:], ALL_cat[j,:])[0,1]
    
    cc = np.corrcoef(ALL_cat)
    
    
    

    FEF_ind = np.where(ALL_ind_roi == 0 )
    FEF_means = []
    FEF_stdevs = []

    VIP_means = []
    VIP_stdevs = []

    for i in range(1,8):
        FEF_means.append(np.mean(c[(ALL_ind_roi == 0) & (ALL_ind_lay == i )]))
        FEF_stdevs.append(np.mean(c[(ALL_ind_roi == 0) & (ALL_ind_lay == i )]))

        VIP_means.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i )]))
        VIP_stdevs.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i )]))


    FEF_means = np.array(FEF_means)
    FEF_stdevs = np.array(FEF_stdevs)

    VIP_means = np.array(VIP_means)
    VIP_stdevs = np.array(VIP_stdevs)





    fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(10,6), layout="constrained")

    p = axs[0, 0].imshow(cc)
    #plt.colorbar()
    fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)



    axs[0, 1].plot(FEF_means)
    axs[0, 1].fill_between(range(7),FEF_means-FEF_stdevs,FEF_means+FEF_stdevs,alpha=.1)

    axs[0, 2].plot(VIP_means)
    axs[0, 2].fill_between(range(7),VIP_means-VIP_stdevs,VIP_means+VIP_stdevs,alpha=.1)

    axs[0,1].title.set_text("FEF")
    axs[0,2].title.set_text("VIP")

    axs[0,1].set(ylabel="r-values")
    axs[0,2].set(ylabel="r-values")

    fig.savefig(plot_dir+"/"+plot_name)
    



seed = FEF_l1_mean
plot_name ='seed_L1-FEF-VIP.png'
seed_valuation_plots(seed, ALL_cat, ALL_ind_roi, ALL_ind_lay, plot_name)


In [ ]:


def seed_valuation_plots(seed, ALL_cat, ALL_ind_roi, ALL_ind_lay, plot_name):


    ALL_height = ALL_cat.shape[0]
    c = np.zeros((ALL_height))
    for i in range(ALL_height):
        c[i] = np.corrcoef(seed, ALL_cat[i,:])[0,1]
    
    
    
    
    
    # ALL_height = ALL_cat.shape[0]
    # cc = np.zeros((ALL_height, ALL_height))
    # for i in range(ALL_height):
    #     for j in range(ALL_height):
    #         cc[i,j] = np.corrcoef(ALL_cat[i,:], ALL_cat[j,:])[0,1]
    
    cc = np.corrcoef(ALL_cat)
    
    
    

    FEF_ind = np.where(ALL_ind_roi == 0 )
    FEF_means = []
    FEF_stdevs = []

    VIP_means = []
    VIP_stdevs = []

    for i in range(1,8):
        FEF_means.append(np.mean(c[(ALL_ind_roi == 0) & (ALL_ind_lay == i )]))
        FEF_stdevs.append(np.mean(c[(ALL_ind_roi == 0) & (ALL_ind_lay == i )]))

        VIP_means.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i )]))
        VIP_stdevs.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i )]))


    FEF_means = np.array(FEF_means)
    FEF_stdevs = np.array(FEF_stdevs)

    VIP_means = np.array(VIP_means)
    VIP_stdevs = np.array(VIP_stdevs)





    fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(10,6), layout="constrained")

    p = axs[0, 0].imshow(cc)
    #plt.colorbar()
    fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)



    axs[0, 1].plot(FEF_means)
    axs[0, 1].fill_between(range(7),FEF_means-FEF_stdevs,FEF_means+FEF_stdevs,alpha=.1)

    axs[0, 2].plot(VIP_means)
    axs[0, 2].fill_between(range(7),VIP_means-VIP_stdevs,VIP_means+VIP_stdevs,alpha=.1)

    axs[0,1].title.set_text("FEF")
    axs[0,2].title.set_text("VIP")

    axs[0,1].set(ylabel="r-values")
    axs[0,2].set(ylabel="r-values")

    fig.savefig(plot_dir+"/"+plot_name)
    



seed = FEF_l1_mean
plot_name ='seed_L1-FEF-VIP.png'
seed_valuation_plots(seed, ALL_cat, ALL_ind_roi, ALL_ind_lay, plot_name)


In [ ]:



ALL_height = ALL_cat.shape[0]
c = np.zeros((ALL_height))
for i in range(ALL_height):
    c[i] = np.corrcoef(seed, ALL_cat[i,:])[0,1]


# ALL_height = ALL_cat.shape[0]
# cc = np.zeros((ALL_height, ALL_height))
# for i in range(ALL_height):
#     for j in range(ALL_height):
#         cc[i,j] = np.corrcoef(ALL_cat[i,:], ALL_cat[j,:])[0,1]



# get PCs reconstruct 
# regress out average 

cc = np.corrcoef(ALL_cat)




FEF_ind = np.where(ALL_ind_roi == 0 )
FEF_means = []
FEF_stdevs = []

VIP_means = []
VIP_stdevs = []

for i in range(1,8):
    FEF_means.append(np.mean(c[(ALL_ind_roi == 0) & (ALL_ind_lay == i )]))
    FEF_stdevs.append(np.mean(c[(ALL_ind_roi == 0) & (ALL_ind_lay == i )]))

    VIP_means.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i )]))
    VIP_stdevs.append(np.mean(c[(ALL_ind_roi == 1) & (ALL_ind_lay == i )]))


FEF_means = np.array(FEF_means)
FEF_stdevs = np.array(FEF_stdevs)

VIP_means = np.array(VIP_means)
VIP_stdevs = np.array(VIP_stdevs)





fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(10,6), layout="constrained")

p = axs[0, 0].imshow(cc)
#plt.colorbar()
fig.colorbar(p, ax=axs[0,0], location='right', anchor=(0, 0.3), shrink=0.7)



axs[0, 1].plot(FEF_means)
axs[0, 1].fill_between(range(7),FEF_means-FEF_stdevs,FEF_means+FEF_stdevs,alpha=.1)

axs[0, 2].plot(VIP_means)
axs[0, 2].fill_between(range(7),VIP_means-VIP_stdevs,VIP_means+VIP_stdevs,alpha=.1)

axs[0,1].title.set_text("FEF")
axs[0,2].title.set_text("VIP")

axs[0,1].set(ylabel="r-values")
axs[0,2].set(ylabel="r-values")

fig.savefig(plot_dir+"/"+plot_name)




seed = FEF_l1_mean
plot_name ='seed_L1-FEF-VIP.png'
seed_valuation_plots(seed, ALL_cat, ALL_ind_roi, ALL_ind_lay, plot_name)



In [ ]:
# FEF_height = FEF_cat.shape[0]
# c = np.zeros((FEF_height))
# for i in range(FEF_height):
#     c[i] = np.corrcoef(FEF_l1_mean, FEF_cat[i,:])[0,1]
    
    





fig, axs = plt.subplots(ncols=len_x, nrows=len_y, figsize=(30,30), #len_x,len_y
                        layout="constrained")
for seed_i in range(len_x):
    seed_lab = labs[seed_i]

    dfs = glob(dataframe_dir+"/*"+seed_lab+"*")
    dfs = [df for df in dfs if "SMOOTH" not in df ]

    nulls = [ df for df in dfs if "perm" in df ] 
    emps   = [ df for df in dfs if "perm" not in df ]

    for target_i in range(len_y):         
        target_lab = labs[target_i]
        target_id = inv_lab_dict[target_lab]
        null_vals, emp_vals  = [], [] 

        for df in nulls: 
            d = np.load(df) 
            d = d[:,1:]
            null_vals.append(d[target_id, : ])
        for df in emps: 
            d = np.load(df) 
            d = d[:,1:]
            emp_vals.append(d[target_id, : ])

        null_vals   = np.stack(null_vals)
        emp_vals    = np.stack(emp_vals) 

        null_means  = null_vals.mean(axis=0)
        null_stdevs  = null_vals.std(axis=0)

        emp_means   = emp_vals.mean(axis=0)
        emp_stdevs  = emp_vals.std(axis=0)

        axs[seed_i, target_i].plot(null_means)
        axs[seed_i, target_i].plot(emp_means)

        axs[seed_i, target_i].fill_between(range(7),null_means-null_stdevs,null_means+null_stdevs,alpha=.1)
        axs[seed_i, target_i].fill_between(range(7),emp_means-emp_stdevs,emp_means+emp_stdevs,alpha=.1)

        if seed_i == 0: 
            axs[seed_i, target_i].title.set_text(target_lab)

        if target_i == 0: 
            axs[seed_i, target_i].set(ylabel=seed_lab)

        
        #axs[seed_i, target_i].legend(['null', 'vaso'])

# ax_i = 0 
# for ax in axs.flat:
#     ax.set(ylabel=labs[ax_i]) #xlabel='x-label', 

# Hide x labels and tick labels for top plots and y ticks for right plots.
# for ax in axs.flat:
#     ax.label_outer()
      
fig.suptitle("V4abstract (Batch20 10iters)", fontsize=16)

fig.savefig(plot_dir+"/V4fig.png")







fig, axs = plt.subplot(rows=1, columns=3, figsize=(3,10))

axs[0,0] = plt.plot()
axs[0,0] = plt.plot()
axs[0,0] = plt.plot()




In [ ]:
print

In [ ]:
seed_path           = roi_dir+"/LIPd_layers.nii.gz"
target_path         = roi_dir+"/FEF_layers.nii.gz"
epi_path            = data_dir+"/VASO_grandmean.nii"

seed_layers         = nib.load(seed_path)
target_layers       = nib.load(target_path)
epi                 = nib.load(epi_path)

seed_layer_data     = seed_layers.get_fdata()
target_layer_data   = target_layers.get_fdata()
epi_data            = epi.get_fdata() 

In [ ]:
unq_lay = np.unique(seed_layers)
seed_voxs = []
for l in unq_lay: 
    l_ind = np.where(seed_layers == l )
    l_voxs = epi_data[l_ind]
    seed_voxs.append(l_voxs)
    

unq_lay = np.unique(seed_layers)
seed_voxs = []
for l in unq_lay: 
    l_ind = np.where(seed_layers == l )
    l_voxs = epi_data[l_ind]
    seed_voxs.append(l_voxs)
    
    
    
    
    
    

In [ ]:
seed_epi_ts = []
seed_epi_prep_ts = [] 
seed_vox_count = []
seed_idx = []

for i in range(1,8):
    idx = np.where(seed_data == i )
    c = len(idx[0])
    
    print("layer {}: {} vox".format(i, c))
    
    seed_epi_ts.append(epi_data[idx])
    #seed_epi_prep_ts.append(epi_prep_data[idx])
    seed_vox_count.append(c)
    seed_idx.append([i]*c)

seed_epi_prep_ts    = np.concatenate(seed_epi_prep_ts)
seed_epi_ts         = np.stack(seed_epi_prep_ts)
seed_idx            = np.stack(seed_epi_prep_ts)

print(seed_epi_prep_ts.shape)
print(seed_epi_ts.shape)
print(seed_idx.shape)


target_epi_ts = []
target_epi_prep_ts = [] 
target_vox_count = []
target_idx = []
for i in range(1,8):
    idx = np.where(target_data == i )
    c = len(idx[0])
    
    print("layer {}: {} vox".format(i, c))
    
    target_epi_ts.append(epi_data[idx])
    #target_epi_prep_ts.append(epi_prep_data[idx])
    target_vox_count.append(c)
    target_idx.append([i]*c)
    
target_epi_prep_ts    = np.concatenate(target_epi_prep_ts)
target_epi_ts         = np.stack(target_epi_prep_ts)
target_idx            = np.stack(target_epi_prep_ts)

print(target_epi_prep_ts.shape)
print(target_epi_ts.shape)
print(target_idx.shape)
